# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection command:
indy0: False


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [4]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (0,0.2,0), 
                           rpy=(np.pi/2,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.2,-0.5,0.),rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (-0.2,-0.5,0), rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", (0.1,0.5,0.01), (0,-0.5,0), rpy=(0,0,np.pi/2), 
                           color=(1,0.7,0.7,0.5), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (-0.4,-0.3,0), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

In [7]:
from pkg.utils.code_scraps import add_indy_gripper_asm2
add_indy_gripper_asm2(gscene, "indy0")

DeprecationWarning: add_indy_gripper_asm2 is deprecated. Set RobotType to indy7gripper instead.

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.2,0.07,0.02), 
                   center=(0,-0.5,0.045), rpy=(0,0,np.pi/2), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.2,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

TypeError: create_binder() takes at least 5 arguments (4 given)

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_subject(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})

### planning pipeline

In [16]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

AssertionError: The geometries of action points should be in the family of subject geometry

## ui

In [17]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

NameError: name 'ppline' is not defined

```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## Test plan

In [18]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.initialize_state(crob.home_pose)

In [20]:
# goal_nodes = [("goal",)]
goal_nodes = [("goal", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=False)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

try: 0 - ('floor', 0)->('grip0', 0)
try transition motion
transition motion tried: False
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
try transition motion
transition motion tried: False
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
try transition motion
transition motion tried: False
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
try transition motion
transition motion tried: False
result: 0 - ('

## play searched plan

In [21]:
ppline.play_schedule(snode_schedule, period=0.01)